In [ ]:
# # This mounts your Google Drive to the Colab VM.
# from google.colab import drive
# drive.mount('/content/drive')

# # TODO: Enter the foldername in your Drive where you have saved the unzipped
# # assignment folder, e.g. 'cs231n/assignments/assignment1/'
# FOLDERNAME = 'cs231n/assignments/assignment1/'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# # Now that we've mounted your Drive, this ensures that
# # the Python interpreter of the Colab VM can load
# # python files from within it.
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# # This downloads the CIFAR-10 dataset to your Drive
# # if it doesn't already exist.
# %cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
# !bash get_datasets.sh
# %cd /content/drive/My\ Drive/$FOLDERNAME

# k-近邻算法 (kNN) 练习

*请完成并提交这份完整的作业工作表（包括输出和任何工作表之外的辅助代码）。更多详情请查看课程网站上的[作业页面](http://vision.stanford.edu/teaching/cs231n/assignments.html)。*

kNN 分类器包含两个阶段：

- 在训练期间，分类器接收训练数据并简单地记住它
- 在测试期间，kNN 通过与所有训练图像进行比较并将 k 个最相似的训练样本的标签进行传递来分类每个测试图像
- k 的值通过交叉验证确定

在这个练习中，你将实现这些步骤并理解基本的图像分类流程、交叉验证，以及提高编写高效、向量化代码的熟练度。

In [ ]:
# 这段代码为notebook环境进行了一些必要的设置和初始化。

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# 让matplotlib生成的图像直接显示在notebook内，而不是弹出新窗口
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # 设置绘图区的默认大小
plt.rcParams['image.interpolation'] = 'nearest' # 显示图片时最近邻插值
plt.rcParams['image.cmap'] = 'gray'            # 设置默认的灰度色表

# 下面两行代码可以让notebook里的外部python模块（比如你自己写的代码文件）代码发生变动时自动重载
%load_ext autoreload
%autoreload 2

In [ ]:
# 加载原始 CIFAR-10 数据。
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

# 清理变量以防止多次加载数据（可能导致内存问题）
try:
   del X_train, y_train
   del X_test, y_test
   print('清除之前加载的数据。')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# 作为健全性检查，我们打印出训练和测试数据的大小。
print('训练数据维度: ', X_train.shape)
print('训练标签维度: ', y_train.shape)
print('测试数据维度: ', X_test.shape)
print('测试标签维度: ', y_test.shape)

In [ ]:
# 可视化数据集中的一些示例。
# 我们展示来自每个类别的几个训练图像示例。
classes = ['飞机', '汽车', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '卡车']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# 为使代码执行更高效，对数据进行子采样
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# 将图像数据重新整形为行
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from cs231n.classifiers import KNearestNeighbor

# 创建一个 kNN 分类器实例。
# 请记住，训练一个 kNN 分类器是一个空操作：
# 分类器只是记住数据，不做进一步处理
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

现在我们想用 kNN 分类器对测试数据进行分类。回想一下，我们可以将这个过程分为两个步骤：

1. 首先我们必须计算所有测试样本和所有训练样本之间的距离
2. 给定这些距离后，对于每个测试样本，我们找到 k 个最近的样本并让它们投票决定标签

让我们从计算所有训练和测试样本之间的距离矩阵开始。例如，如果有 **Ntr** 个训练样本和 **Nte** 个测试样本，这个阶段应该产生一个 **Nte × Ntr** 的矩阵，其中每个元素 (i,j) 是第 i 个测试样本和第 j 个训练样本之间的距离。

**注意：对于这个 notebook 中要求的三个距离计算，你不能使用 numpy 提供的 np.linalg.norm() 函数。**

首先，打开 `cs231n/classifiers/k_nearest_neighbor.py` 并实现函数 `compute_distances_two_loops`，该函数使用（非常低效的）双循环来遍历所有（测试，训练）样本对，一次计算一个距离矩阵元素。

In [ ]:
# 打开 cs231n/classifiers/k_nearest_neighbor.py 并实现
# compute_distances_two_loops。

# 测试你的实现：
dists = classifier.compute_distances_two_loops(X_test)
print(dists.shape)

In [ ]:
# 我们可以可视化距离矩阵：每一行是一个测试样本和它与训练样本的距离
plt.imshow(dists, interpolation='none')
plt.show()

**内联问题 1**

注意到距离矩阵中有一些结构化模式，其中一些行或列明显更亮。（注意，在默认的配色方案中，黑色表示低距离，而白色表示高距离。）

- 数据中是什么原因导致了明显亮的行？
- 什么原因导致了列？

$\color{blue}{\\textit 你的答案：}$ *在此填写。*

In [ ]:
# 现在实现函数 predict_labels 并运行下面的代码：
# 我们使用 k = 1（即最近邻）。
y_test_pred = classifier.predict_labels(dists, k=1)

# 计算并打印正确预测的样本比例
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('正确 %d / %d => 准确率: %f' % (num_correct, num_test, accuracy))

你应该期望看到大约 `27%` 的准确率。现在让我们尝试一个更大的 `k`，比如 `k = 5`：

In [ ]:
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('正确 %d / %d => 准确率: %f' % (num_correct, num_test, accuracy))

你应该期望看到比 `k = 1` 略好的性能。

**内联问题 2**

我们也可以使用其他距离度量，如 L1 距离。
对于某个图像 $I_k$ 在位置 $(i,j)$ 处的像素值 $p_{ij}^{(k)}$，

所有图像在所有像素上的平均值是 $$\mu=\frac{1}{nhw}\sum_{k=1}^n\sum_{i=1}^{h}\sum_{j=1}^{w}p_{ij}^{(k)}$$
所有图像在像素 $(i,j)$ 处的平均值是
$$\mu_{ij}=\frac{1}{n}\sum_{k=1}^np_{ij}^{(k)}.$$
标准差 $\sigma$ 和像素级标准差 $\sigma_{ij}$ 的定义类似。

以下哪些预处理步骤不会改变使用 L1 距离的最近邻分类器的性能？选择所有适用的选项。为了澄清，训练和测试样本都使用相同的方式进行预处理。

1. 减去平均值 $\mu$ ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu$。)
2. 减去每个像素的平均值 $\mu_{ij}$  ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu_{ij}$。)
3. 减去平均值 $\mu$ 并除以标准差 $\sigma$。
4. 减去像素级平均值 $\mu_{ij}$ 并除以像素级标准差 $\sigma_{ij}$。
5. 旋转数据坐标轴，这意味着将所有图像旋转相同的角度。旋转引起的图像空白区域用相同的像素值填充，不进行插值。

$\color{blue}{\\textit 你的答案：}$

$\color{blue}{\\textit 你的解释：}$

In [ ]:
# 现在让我们通过使用部分向量化和一个循环来加速距离矩阵计算
# 实现函数 compute_distances_one_loop 并运行下面的代码：
dists_one = classifier.compute_distances_one_loop(X_test)

# 为了确保我们的向量化实现是正确的，我们确保它与朴素实现一致。有很多方法可以决定两个矩阵是否相似；其中最简单的是 Frobenius 范数。如果你之前没有见过它，两个矩阵的 Frobenius 范数是所有元素差值平方和的平方根；换句话说，将矩阵重塑为向量并计算它们之间的欧几里得距离。
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('单循环版本的差异: %f' % (difference, ))
if difference < 0.001:
    print('很好！距离矩阵是相同的')
else:
    print('糟糕！距离矩阵不同')

In [ ]:
# 现在在 compute_distances_no_loops 内实现完全向量化的版本并运行代码
dists_two = classifier.compute_distances_no_loops(X_test)

# 检查距离矩阵是否与我们之前计算的一致：
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('无循环版本的差异: %f' % (difference, ))
if difference < 0.001:
    print('很好！距离矩阵是相同的')
else:
    print('糟糕！距离矩阵不同')

In [ ]:
# 让我们比较实现的运行速度
def time_function(f, *args):
    """
    调用函数 f 和参数，并返回执行所需的时间（以秒为单位）。
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('双循环版本用时 %f 秒' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('单循环版本用时 %f 秒' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('无循环版本用时 %f 秒' % no_loop_time)

# 你应该看到完全向量化的实现有显著的性能提升！

# 注意：这取决于你使用的机器，
# 你可能不会看到从两个循环到一个循环时的加速，
# 甚至可能会看到减速。

### 交叉验证

我们已经实现了 k-近邻分类器，但我们任意设置了 k = 5 的值。现在我们将通过交叉验证来确定这个超参数的最佳值。

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
################################################################################
# TODO:                                                                        #
# 将训练数据拆分为折叠。拆分后，X_train_folds 和 y_train_folds 应该          #
# 都是长度为 num_folds 的列表，其中                                         #
# y_train_folds[i] 是 X_train_folds[i] 中点的标签向量。                     #
# 提示：查看 numpy array_split 函数。                                        #
################################################################################


# 一个保存我们找到的不同 k 值的准确率的字典。运行交叉验证后，
# k_to_accuracies[k] 应该是一个长度为 num_folds 的列表，
# 给出使用该 k 值时找到的不同准确率值。
k_to_accuracies = {}


################################################################################
# TODO:                                                                        #
# 执行 k 折交叉验证以找到 k 的最佳值。对于每个可能的 k 值，运行             #
# k-近邻算法 num_folds 次，在每次中除了一个折叠外的所有折叠用作训练数据，   #
# 最后一个折叠用作验证集。将所有折叠和所有 k 值的准确率存储在              #
# k_to_accuracies 字典中。                                                     #
################################################################################


# 打印出计算出的准确率
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, 准确率 = %f' % (k, accuracy))

In [ ]:
# 绘制原始观测值
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# 绘制带误差线的趋势线，对应标准差
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('k 的交叉验证')
plt.xlabel('k')
plt.ylabel('交叉验证准确率')
plt.show()

In [ ]:
# 基于上面的交叉验证结果，选择 k 的最佳值，重新训练分类器
# 使用所有训练数据，并在测试数据上进行测试。你应该能够获得
# 超过 28% 的测试准确率。
best_k = 1

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred = classifier.predict(X_test, k=best_k)

# 计算并显示准确率
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('正确 %d / %d => 准确率: %f' % (num_correct, num_test, accuracy))

**内联问题 3**

关于 $k$-最近邻（$k$-NN）在分类设置中，以下哪些说法对于所有 $k$ 都是正确的？选择所有适用的选项。

1. $k$-NN 分类器的决策边界是线性的。
2. 1-NN 的训练误差总是小于或等于 5-NN 的训练误差。
3. 1-NN 的测试误差总是小于 5-NN 的测试误差。
4. 使用 $k$-NN 分类器分类一个测试样本所需的时间随着训练集的大小而增长。
5. 以上都不对。

$\color{blue}{\\textit 你的答案：}$

$\color{blue}{\\textit 你的解释：}$